In [1]:
import napari
import PIL.Image
import numpy as np

PIL.Image.MAX_IMAGE_PIXELS = 1056323868

v = napari.Viewer()

# lab = np.zeros((109, 2800, 1987), dtype='int')
# lab[10:90,100:2700,100:1900] = 1
# v.add_labels(lab, scale=(100,3.5,3.5))

In [ ]:
import zarr
from skimage import io
from skimage import measure
from tqdm import tqdm
import h5py
import numpy as np

## convert ilastik output to labels in tif format

In [ ]:


f = h5py.File(
    "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/191815_05401_CH1_Simple Segmentation.h5",
    "r",
)
pred = f.get("exported_data")
pred = pred[:, :, 0]
mask = pred == 1
mask = mask.astype("int32")
labels = measure.label(mask)
io.imsave("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/54_comps.tif", labels)

## write zarr

In [ ]:
zarr_labels = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='i4')

for z in tqdm(range(52, 57)):
    labels = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/" + str(z) + "_comps.tif")
    zarr_labels[z,:,:] = labels

zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/labels_zarr.zarr', zarr_labels)

In [ ]:
zarr_image = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='i4')

for z in tqdm(range(52, 57)):
    image = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/191815_0" + str(z) + "01_CH1.tif")
    zarr_image[z,:,:] = image

#zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr.zarr', zarr_image)

In [19]:
import zarr
zarr_image = zarr.open("/Users/thomasathey/Documents/mimlab/mouselight/BIL/image_labels_partial.zarr")
zarr_image.shape



(109, 28001, 19868)

In [ ]:
for x1 in range(0, 20000,2000):
    x2 = np.amin([x1+2000, zarr_image.shape[2]])
    im = zarr_image[52:57,:,x1:x2]
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/" + str(x1) + ".tif"
    io.imsave(fname, im)

In [ ]:
zarr_probs = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='float')
for x1 in range(0, 20000,2000):
    x2 = np.amin([x1+2000, zarr_image.shape[2]])
    f = h5py.File(
        "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/" + str(x1) + "_Probabilities.h5",
        "r",
    )
    pred = f.get("exported_data")
    probs = pred[:,:,:,0]
    zarr_probs[52:57,:,x1:x2] = probs
    break


zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/probs_zarr.zarr', zarr_probs)

# Viterbrain

In [ ]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

In [ ]:
sg = state_generation(
    image_path="/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr.zarr",
    ilastik_program_path="/Applications/ilastik-1.4.0b21-OSX.app/Contents/ilastik-release/run_ilastik.sh",
    ilastik_project_path="/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/bil_slice.ilp",
    chunk_size=[5,1000,1000],
    soma_coords=[],
    parallel=8,
    prob_path = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/probs_zarr.zarr",
)

In [ ]:
sg.compute_frags()

## Check labels progress

In [5]:
from skimage import io
import napari
import PIL.Image
import numpy as np


im = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/image.tif")
im_probs = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/image_probs.tif")
im_labels = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/image_labels.tif")

print("Displaying")
v = napari.Viewer()
v.add_image(im, name="image", contrast_limits=[0, 2**16])
v.add_image(im_probs, name="probs")
# v.add_labels(im_probs>0.15, name="0.15")
# v.add_labels(im_probs>0.25, name="0.25")
# v.add_labels(im_probs>0.5, name="0.5")
v.add_labels(im_probs>0.9, name="0.9")
v.add_labels(im_labels, name="labels")

Displaying


/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_vispy/layers/image.py:233: UserWarning: data shape (28001, 19868) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_vispy/layers/image.py:233: UserWarning: data shape (28001, 19868) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_vispy/layers/image.py:233: UserWarning: data shape (28001, 19868) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_vispy/layers/image.py:233: UserWarning

<Labels layer 'labels' at 0x14f164fa0>

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_vispy/layers/image.py:233: UserWarning: data shape (28001, 19868) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
